##  Not intended for official purposes. 
## This notebook supports a brainstorming discussion on how to handle static/fixed variables in a way that is interoperable with GFDL and CMIP as we add this feature to the GFDL catalog builder

# Join discussion here 
# https://github.com/NOAA-GFDL/MDTF-diagnostics/discussions/686

### Reference: https://github.com/aradhakrishnanGFDL/canopy-cats

### Contact: Open issues here https://github.com/NOAA-GFDL/catalogbuilder

In [16]:
import xarray as xr
import intake,yaml,os
import intake_esm
import matplotlib
import ast
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# option 1 

In [19]:
col_url = "/home/a1r/github/noaa-gfdl/catalogs/statics_omonthly_opt1.json"

In [21]:
col = intake.open_esm_datastore(col_url,
                                read_csv_kwargs={"converters": {"variable_id": ast.literal_eval}},)

In [22]:
col.esmcat.has_multiple_variable_assets

True

In [25]:
cat_subset = col.search(variable_id=['areacello','sftof'])

In [26]:
dsets = cat_subset.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id.experiment_id.frequency.table_id.realm'


In [28]:
dsets['fre.CM4.5v01_om5b06_piC_noBLING.fx.fx.ocean_monthly']

<xarray.Dataset> Size: 13MB
Dimensions:    (yh: 1161, xh: 1440)
Coordinates:
  * xh         (xh) float64 12kB -299.9 -299.6 -299.4 ... 59.38 59.62 59.88
  * yh         (yh) float64 9kB -88.52 -88.42 -88.32 ... 89.73 89.84 89.95
Data variables:
    areacello  (yh, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    sftof      (yh, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
Attributes: (12/14)
    external_variables:              areacello areacello
    intake_esm_vars:                 ['areacello', 'sftof']
    intake_esm_attrs:activity_id:    dev
    intake_esm_attrs:source_id:      fre
    intake_esm_attrs:experiment_id:  CM4.5v01_om5b06_piC_noBLING
    intake_esm_attrs:frequency:      fx
    ...                              ...
    intake_esm_attrs:variable_id:    areacello,sftof,hfgeou
    intake_esm_attrs:platform:       gfdl.ncrc5-intel23-prod-openmp
    intake_esm_attrs:cell_methods:   ts
    intake_esm_attrs:path:           /archive/John.Krasting/fre/FMS2024.02_OM...
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          fre.CM4.5v01_om5b06_piC_noBLING.fx.fx.oc...

# option 2

# INSERT THE LOCATION OF YOUR DATA CATALOG IN THE CELL BELOW TO EXPLORE SOMETHING ELSE

In [6]:
col_url = "/home/a1r/github/noaa-gfdl/catalogs/statics_omonthly.json"

In [7]:
col = intake.open_esm_datastore(col_url)

#### Explore the catalog by viewing the "data frame" (df)

In [8]:
col.df.head

<bound method NDFrame.head of      activity_id  institution_id source_id                experiment_id  \
0            dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
1            dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
2            dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
3            dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
4            dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
...          ...             ...       ...                          ...   
1109         dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
1110         dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
1111         dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
1112         dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   
1113         dev             NaN       fre  CM4.5v01_om5b06_piC_noBLING   

     frequency          realm table_id  member_id  grid_label  variab

In [9]:
col.df.columns

Index(['activity_id', 'institution_id', 'source_id', 'experiment_id',
       'frequency', 'realm', 'table_id', 'member_id', 'grid_label',
       'variable_id', 'time_range', 'chunk_freq', 'platform', 'dimensions',
       'cell_methods', 'standard_name', 'path'],
      dtype='object')

In [10]:
dict(col.esmcat)["aggregation_control"]

AggregationControl(variable_column_name='variable_id', groupby_attrs=['source_id', 'experiment_id', 'frequency', 'table_id', 'grid_label', 'realm', 'member_id', 'chunk_freq'], aggregations=[Aggregation(type=<AggregationType.union: 'union'>, attribute_name='variable_id', options={}), Aggregation(type=<AggregationType.join_existing: 'join_existing'>, attribute_name='time_range', options={'dim': 'time', 'coords': 'minimal', 'compat': 'override'})])

# Search for statics 

In [24]:
expname_filter = ['CM4.5v01_om5b06_piC_noBLING']
realm = 'ocean_monthly'
variable_id_filter = "wfo"
frequency = "fx"
table_id = "fx"

catstatics = col.search(experiment_id=expname_filter,
                 frequency=frequency,realm=realm,table_id = "fx")
                 #add this if you want to filter by variable ,variable_id = 'areacello')

# Explore data: lets search for variables that I may need for my analysis

In [17]:
#Examples to just search for what we want from the catalog
expname_filter = ['CM4.5v01_om5b06_piC_noBLING']
realm = 'ocean_monthly'
variable_id_filter = "wfo"
frequency = "mon"

In [18]:
cat = col.search(experiment_id=expname_filter,
                 frequency=frequency,realm=realm,
                 variable_id = variable_id_filter)

In [19]:
cat

,unique
activity_id,1
institution_id,0
source_id,1
experiment_id,1
frequency,1
realm,1
table_id,0
member_id,0
grid_label,0
variable_id,1


## If files are in a DMF managed TAPE system, follow organizational and file system best practices. 

If the files are in /archive, use the following dmgetmagic() to bring the files online before proceeding.

In [29]:
#for simple dmget usage, just use this !dmget {file}
#use following to wrap the dmget call for each path in the catalog
def dmgetmagic(x):
    cmd = 'dmget %s'% str(x) 
    return os.system(cmd)

#OR refer to importing dmget ,  https://github.com/aradhakrishnanGFDL/canopy-cats/tree/main/notebooks/dmget.py

In [31]:
dmstatus = cat.df["path"].apply(dmgetmagic)

### to_dataset_dict (cell below): Load catalog entries into a dictionary of xarray datasets.

### Column values, dataset keys and requested variables are added as global attributes on the returned datasets.

In [37]:
dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time':5}, 'decode_times': False})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id.experiment_id.frequency.realm.chunk_freq'


In [39]:
for k in dset_dict.keys(): 
    print(k)

CM4.CM4_piControl_C.mon.atmos_cmip.5yr


In [40]:
ds = dset_dict["CM4.CM4_piControl_C.mon.atmos_cmip.5yr"]

In [41]:
ds #the xarray dataset object

<xarray.Dataset> Size: 2GB
Dimensions:     (time: 7800, lat: 180, lon: 288, bnds: 2)
Coordinates:
    average_DT  (time) float64 62kB dask.array<chunksize=(5,), meta=np.ndarray>
    average_T1  (time) float64 62kB dask.array<chunksize=(5,), meta=np.ndarray>
    average_T2  (time) float64 62kB dask.array<chunksize=(5,), meta=np.ndarray>
  * bnds        (bnds) float64 16B 1.0 2.0
  * lat         (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
    lat_bnds    (lat, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon         (lon) float64 2kB 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
    lon_bnds    (lon, bnds) float64 5kB dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time        (time) float64 62kB 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
    time_bnds   (time, bnds) float64 125kB dask.array<chunksize=(5, 2), meta=np.ndarray>
Data variables:
    rsdt        (time, lat, lon) float32 2GB dask.array<chunksize=(5, 180, 288), meta=np.ndarray>
Attributes: (12/18)
    title:                           CM4_piControl_C
    grid_type:                       regular
    grid_tile:                       N/A
    code_version:                    $Name: bronx-10_performance_z1l $
    external_variables:              area
    intake_esm_vars:                 ['rsdt']
    ...                              ...
    intake_esm_attrs:chunk_freq:     5yr
    intake_esm_attrs:platform:       gfdl.ncrc4-intel16-prod-openmp
    intake_esm_attrs:cell_methods:   ts
    intake_esm_attrs:standard_name:  toa_incoming_shortwave_flux
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          CM4.CM4_piControl_C.mon.atmos_cmip.5yr

In [42]:
ds.rsdt #data array

<xarray.DataArray 'rsdt' (time: 7800, lat: 180, lon: 288)> Size: 2GB
dask.array<concatenate, shape=(7800, 180, 288), dtype=float32, chunksize=(5, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    average_DT  (time) float64 62kB dask.array<chunksize=(5,), meta=np.ndarray>
    average_T1  (time) float64 62kB dask.array<chunksize=(5,), meta=np.ndarray>
    average_T2  (time) float64 62kB dask.array<chunksize=(5,), meta=np.ndarray>
  * lat         (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon         (lon) float64 2kB 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
  * time        (time) float64 62kB 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
Attributes:
    long_name:      TOA Incident Shortwave Radiation
    units:          W m-2
    cell_methods:   time: mean
    cell_measures:  area: area
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  toa_incoming_shortwave_flux
    interp_method:  conserve_order1

# option 3

In [30]:
col_url = "/home/a1r/github/noaa-gfdl/catalogs/statics_omonthly_opt3.json"

In [31]:
col = intake.open_esm_datastore(col_url)

In [33]:
expname_filter = ['CM4.5v01_om5b06_piC_noBLING']
realm = 'ocean_monthly'
frequency = "fx"
table_id = "fx"

cat_subset = col.search(experiment_id=expname_filter,
                 frequency=frequency,realm=realm,table_id = "fx")


In [34]:
dset_dict = cat_subset.to_dataset_dict(cdf_kwargs={'chunks': {'time':5}, 'decode_times': False})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id.experiment_id.frequency.table_id.realm'


In [36]:
dset_dict['fre.CM4.5v01_om5b06_piC_noBLING.fx.fx.ocean_monthly']

<xarray.Dataset> Size: 174MB
Dimensions:       (yq: 1161, xq: 1440, yh: 1161, xh: 1440)
Coordinates:
  * xh            (xh) float64 12kB -299.9 -299.6 -299.4 ... 59.38 59.62 59.88
  * xq            (xq) float64 12kB -299.8 -299.5 -299.2 ... 59.5 59.75 60.0
  * yh            (yh) float64 9kB -88.52 -88.42 -88.32 ... 89.73 89.84 89.95
  * yq            (yq) float64 9kB -88.47 -88.37 -88.28 ... 89.78 89.89 90.0
Data variables: (12/26)
    Coriolis      (yq, xq) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    areacello     (yh, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    areacello_bu  (yq, xq) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    areacello_cu  (yh, xq) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    areacello_cv  (yq, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    deptho        (yh, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    ...            ...
    hfgeou        (yh, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    sftof         (yh, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    wet           (yh, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    wet_c         (yq, xq) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    wet_u         (yh, xq) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
    wet_v         (yq, xh) float32 7MB dask.array<chunksize=(1161, 1440), meta=np.ndarray>
Attributes: (12/14)
    external_variables:              areacello areacello
    intake_esm_vars:                 ['fixed']
    intake_esm_attrs:activity_id:    dev
    intake_esm_attrs:source_id:      fre
    intake_esm_attrs:experiment_id:  CM4.5v01_om5b06_piC_noBLING
    intake_esm_attrs:frequency:      fx
    ...                              ...
    intake_esm_attrs:variable_id:    fixed
    intake_esm_attrs:platform:       gfdl.ncrc5-intel23-prod-openmp
    intake_esm_attrs:cell_methods:   ts
    intake_esm_attrs:path:           /archive/John.Krasting/fre/FMS2024.02_OM...
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          fre.CM4.5v01_om5b06_piC_noBLING.fx.fx.oc...

In [37]:
dset_dict['fre.CM4.5v01_om5b06_piC_noBLING.fx.fx.ocean_monthly'].sftof

<xarray.DataArray 'sftof' (yh: 1161, xh: 1440)> Size: 7MB
dask.array<open_dataset-sftof, shape=(1161, 1440), dtype=float32, chunksize=(1161, 1440), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 12kB -299.9 -299.6 -299.4 -299.1 ... 59.38 59.62 59.88
  * yh       (yh) float64 9kB -88.52 -88.42 -88.32 -88.23 ... 89.73 89.84 89.95
Attributes:
    units:          %
    long_name:      Sea Area Fraction
    cell_methods:   area:mean yh:mean xh:mean time: point
    standard_name:  SeaAreaFraction

In [39]:
expname_filter = ['CM4.5v01_om5b06_piC_noBLING']
realm = 'ocean_monthly'
frequency = "mon"
variable_id = "hfsnthermds"
cat_subset = col.search(experiment_id=expname_filter,
                 frequency=frequency,realm=realm,variable_id=variable_id)

In [40]:
dset_dict = cat_subset.to_dataset_dict(cdf_kwargs={'chunks': {'time':5}, 'decode_times': False})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id.experiment_id.frequency.realm.chunk_freq'


In [42]:
dset_dict['fre.CM4.5v01_om5b06_piC_noBLING.mon.ocean_monthly.5yr']

<xarray.Dataset> Size: 401MB
Dimensions:      (time: 60, yh: 1161, xh: 1440, nv: 2)
Coordinates:
  * nv           (nv) float64 16B 1.0 2.0
  * time         (time) float64 480B 15.5 45.0 74.5 ... 1.779e+03 1.81e+03
  * xh           (xh) float64 12kB -299.9 -299.6 -299.4 ... 59.38 59.62 59.88
  * yh           (yh) float64 9kB -88.52 -88.42 -88.32 ... 89.73 89.84 89.95
Data variables:
    average_DT   (time) float64 480B dask.array<chunksize=(5,), meta=np.ndarray>
    average_T1   (time) float64 480B dask.array<chunksize=(5,), meta=np.ndarray>
    average_T2   (time) float64 480B dask.array<chunksize=(5,), meta=np.ndarray>
    hfsnthermds  (time, yh, xh) float32 401MB dask.array<chunksize=(5, 1161, 1440), meta=np.ndarray>
    time_bnds    (time, nv) float64 960B dask.array<chunksize=(5, 2), meta=np.ndarray>
Attributes: (12/20)
    title:                           CM4.5v01_om5b06_piC_noBLING
    associated_files:                areacello: 00010101.ocean_static.nc
    grid_type:                       regular
    grid_tile:                       N/A
    external_variables:              areacello
    intake_esm_vars:                 ['hfsnthermds']
    ...                              ...
    intake_esm_attrs:platform:       gfdl.ncrc5-intel23-prod-openmp
    intake_esm_attrs:cell_methods:   ts
    intake_esm_attrs:standard_name:  heat_flux_into_sea_water_due_to_snow_the...
    intake_esm_attrs:path:           /archive/John.Krasting/fre/FMS2024.02_OM...
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          fre.CM4.5v01_om5b06_piC_noBLING.mon.ocea...